In [141]:
# HE53_Bootstrap
# Jesse Bausell
# 18 September 2019
#
# HE53_Bootstrap enables user to evaluate radiative transfer using field-measured inherent optical properties (IOPs). 
# The program accepts ac-s and hs6 data and performs bootstrapping with replacement to produce a series of depth-binned
# absorption, attenuation, and backscattering from single spectra. The resulting output files can be uploaded into 
# Hydrolight formodeling radiative transfer. Program is specifically designed to work with Seabird Scientific ac-s 
# meter and HOBI Labs Hydroscat6 meter. See readme for more details.
#
# Inputs: 
    # ac-s_Data - single absorption/attenuation spectra. Program accepts ascii or hdf5/mat files.
    # hs6_Data - single particulate backscattering spectra. Program accepts ascii or hdf5/mat files
    # UserTemplate_ascii.txt - ascii file with user-specified data. Template provided.
# Outputs:
    # STATION_ac-s_bin##_itr#.txt - hydrolight compatible depth-binned absorption/attenuation
    # STATION_hs6_bin##_itr#.txt - hydrolight compatible depth-binned particulate backscattering
    # STATION_bin##_itr#.txt - # batch file used to run hydrolight using aforementioned depth-binned IOPs
# Please note:
# Bootstrapping with replacement is repeated n times
# ##- denotes user-specified depth bin size
#  #- denotes bootstrap iteration (e.g. 1, 2, 3 ... n)

In [142]:
## Import python libraries
import tkinter as tk # Import the proper libraries for inserting a file
from tkinter import filedialog # Filedialog allows user to select an input file
from datetime import datetime as dt # 
import numpy as np # Import the numpy as np
import h5py # Import hdf library (used for mat/dfh5 files)
import re # Import regular expression library
import csv # Import csv reader/writer library
import os # Import miscellaneous operating systems interface library

In [143]:
## Import user-defined functions

In [144]:
def UserTemplate(file_path):
    """Opens user template and places input data into a python dictionary.
    Inputs: 
        file_path - file pathway and name for header file (see readme)
    Outputs:
        User_Dict - python dictionary containing all user-supplied specifications"""
    keY = 0 # Key to regulate while loop
    User_Dict = {} # Create dictionary for user-input data
    with open(file_path) as raw: # Open User template 
        while 1:
            # This while loop cycles through the header file lines, collects important info, 
            # and puts it into a dictionary
            title = raw.readline()
            if 'Station Name' in title:
                # For the first line of important information
                keY += 1 # increase keY by one.
            if keY > 0:
                # If the first line of important information has been reached, do these steps 
                # for each subsequent while-loop iteration...
                IND = title.index(':') # Find index of ':'
                User_Dict[title[:IND]] = title[IND+2:-1] # Take information right of ":". Exclude "\n"
                # and put it into a dictionary
                if 'Bootstrap Iterations' in title:
                    # Once the last important header line is reached, break the while-loop
                    break
        return(User_Dict)
             

In [145]:
def acs_HFD_LOADER(hdf_name):
    """This function loads processed ac-s data (individual spectra) from .mat/hdf5 files 
    that will later be bootstrapped. It is designed for data prepared by the matlab program 
    acsPROCESS_INTERACTIVE.m
    Inputs:
        hdf_name - name of hdf file
    Outputs
        A_CORR - absorption data
        C_CORR - attenuation data
        deptH - ac-s depths
        lambdA - ac-s wavelengths"""
    with h5py.File(hdf_name) as hf:
        # Assign variables to hdf fields
        A_CORR = hf['A_CORR'][()] # Absorption values
        C_CORR = hf['C_CORR'][()] # Attenuation values
        deptH = hf['deptH'][()] # Depth values
        acs_wvl = hf['lambda'][()] # Wavelengths
        # Transpose matrices 
        A_CORR = np.transpose(A_CORR) # Transpose absorption
        C_CORR = np.transpose(C_CORR) # Transpose attenuation
        # Simplify depth and wavelength arrays
        deptH = deptH[0] # Make depth array one dimensional
        lambdA = np.zeros(len(acs_wvl)) # Create one dimensional array of zeros the same size as the number of wavelengths
        for i,j in enumerate(acs_wvl):
            # For-loop assigns a wavelength to each zero value in the newly-created 'lambdA array (of zeros)
            lambdA[i] = j[0] # index and assign wavelengths
        return(A_CORR,C_CORR,deptH,lambdA)

def SEABASS_TXT_LOADER(txt_name):
    """This function loads processed ac-s or hs6 data from Seabass-formatted .txt files.
    It is designed for data prepared by the matlab programs acsPROCESS_SEABASS or hs6PROCESS_SEABAS.
    It is theoretically compatible with HOBI Labs hs4 and hs2 meters, but has not been tested. User should 
    keep in mind that when using hs6 (backscattering) data, A_CORR and C_CORR output variables are redundant.
        Inputs:
        hdf_name - name of hdf file
    Outputs
        A_CORR - absorption or backscattering data
        C_CORR - attenuation or backscattering data
        deptH - ac-s or hs6 depths
        lambdA - ac-s or hs6 wavelengths"""
    with open(txt_name) as raw: # Open user-input SEABASS.txt file
        keY = 0 # key variable to distinguish header from data
        for title in raw:
            # This for-loop reads every line of the SEABASS.txt file one by one
            if 'agp' in title: 
                # SEABASS.txt is an ac-s file, find the header line with column fields
                fielDS = title[:-1] # Remove end-of-line character
                IND_c = fielDS.find(',cgp') # Find the start of attenuation (cgp) column headers
                markeR1 = ',agp' # create marker for absorption (agp) column headers
                markeR2 = ',cgp' # create marker for attenuation (cgp) column headers
                lambdA_stra = fielDS[:IND_c] # Isolate the section of the header line with only absorpiton fields
                lambdA_strc = fielDS[IND_c:] # Isolate the section of the header line with only attenuation fields
            elif 'depth,bbp' in title:
            # SEABASS.txt is an hs6 file, find the header line with column fields
                fielDS = title[:-1] # Remove end-of-line character
                IND_c = fielDS.find(',stimf') # Find the start of backscattering (bbp) column headers
                markeR1 = ',bbp' # create marker for backscattering column headers
                markeR2 = ',bbp' # make secone marker backscattering column headers
                lambdA_stra = fielDS[:IND_c] # Isolate the section of the header line with only backscattering fields
                lambdA_strc = fielDS[:IND_c] # Provide redundant duplicate to reduce code
            elif 'end_header' in title:
            # The final line of code has been reached
                lambdA_a = lambdA_stra.split(markeR1) # Create a list of absorption (or backscttering) wavelengths 
                lambdA_a.pop(0) # Remove the first item from the list (because it's junk)
                lambdA_a = np.asarray(lambdA_a,dtype=np.float) # Convert list into numpy array
                lambdA_c = lambdA_strc.split(markeR2) # Create a list of attenuation (or backscttering) wavelengths
                lambdA_c.pop(0) # Remove the first item from the list (because it's junk)
                lambdA_c = np.asarray(lambdA_c,dtype=np.float) # Convert list into numpy array
                keY +=1 # Because the last line of the metadata has been reached, increase keY variable by 1
                data = dict() # Create dictionary to store data
                fielDS = fielDS.split(',') # Create list of header line column fields            
            elif keY == 1:
            # Header metadata has been processed and the for-loop is examining data
                title = title.split('\t') # split line of data into list
                for t in fielDS:
                    # for-loop takes a line of SEABASS data and organizes it into python dictionary (data)
                    IND = fielDS.index(t) 
                    if t in data.keys():
                        data[t].append(title[IND])
                    else:
                        data[t] = [title[IND]]
    deptH = np.asarray(data['depth'],dtype=np.float) # create array of IOP depth values
    A_CORR = np.zeros([len(deptH),len(lambdA_a)])*np.nan # create nan matrix for absorption (or backscattering) values
    C_CORR = np.zeros([len(deptH),len(lambdA_a)])*np.nan # create nan matrix for attenuation (or redundant backscattering) values
    for i,l in enumerate(lambdA_a):
        # for-loop fills in A_CORR and C_CORR matrices one column at a time. Wavelengths and markers (agp/cgp or bbp) are
        # used to construct dictionary keys and index them onto matrices
        try:
            # If the wavelenght IS NOT an integer, put it directly into the python dictionary and index data (absorption or backscattering)
            A_CORR[:,i] = np.asarray(data[markeR1[1:] + str(l)],dtype=np.float)
        except:
            # If the wavelenght IS an integer but has a decimal of 0, remove .0 and index data (absorption or backscattering)
            A_CORR[:,i] = np.asarray(data[markeR1[1:] + str(int(l))],dtype=np.float)
        try:
            # If the wavelenght IS NOT an integer, put it directly into the python dictionary and index data (attenuation or backscattering)
            C_CORR[:,i] = np.asarray(data[markeR2[1:] + str(lambdA_c[i])],dtype=np.float)
        except:
            # If the wavelenght IS an integer but has a decimal of 0, remove .0 and index data (attenuation or backscattering)
            C_CORR[:,i] = np.asarray(data[markeR2[1:] + str(int(lambdA_c[i]))],dtype=np.float)
    return(A_CORR,C_CORR,deptH,lambdA_a)

In [146]:
def createFolder(directory):
    """ createFolder searches for a dirctory specified by the user. If there is none, it creates one"""
    try:
        if not os.path.exists(directory): # If the folder doesn't exist
            os.makedirs(directory) # Create a folder
    except OSError: # If there is an error other than a non-existant folder
        print ('Error: Creating directory. ' +  directory) # report error and shut down createFolder
        
def headER_reader(hdf5_FILE,**parameters):
    """Opens header template file (hdf5 file) and selects the instrument-specific header template 
    as specified by the user.
    Inputs:
        hdf5_FILE - hdf5 file containing generic header templates for ac-s, hs6, and hydrolight batch files
    Keyword Arguments:
        key - specify the header template to retrieve: "ac-s", "hs6" or "batch" 
    Outputs:
        headER_final - python dictionary containing user-specified header template"""
    key = parameters['key'] # Define user-specified keyword argument
    with h5py.File(hdf5_FILE) as hf:
        # Open header file
        headER = hf[key] # Assign dictionary-like data to a variable using the keyword argument
        headER_final = {} # Create a blank dictionary for output variable
        for i in headER:
            # for-loop cycles through each hdf5 dictionary key and places data into an actual dictionary
            headER_final[i] = headER[i][()] # Create dictionary element
        return(headER_final) # Output final instrument-secific dictionary with generic header template information

def Boostrap_FileWriter(BOOTSRAP_MATRIX,User_Dict,**parameters):
    """Creates a Hydrolight-compatible ascii file using depth-binned IOP data. File is output in a separate folder.
    This folder is placed in the same directory as the original file containing non-binned IOP data.
    Inputs:
        BOOTSRAP_MATRIX - Depth-binned IOP matrix
        User_Dict - Python dictionary containing user-specified inputs
    keyword arguments:
        itR - specify bootstrap iteration. Must be an integer.
        wvl - numpy array of IOP wavelengths.
        INSTRUMENT - specify whether IOP matrix is 'acs' or 'hs6'
    Outputs:
        Hyrolight-compatible file of binned IOPs (not a variable)"""
    ### First, import appropriate header template (instrument-dependent)
    inST = parameters['INSTRUMENT'] # specify instrument name
    headER = headER_reader('HE53_Bootstrap_metadata1',key=inST) # Find header metadata template for the appropriate instrument (ac-s or hs6)
    ### Second, define user-supplied variables INDEPENDENT of instrument type
    ## 2a. define keyword arguments
    itR = str(parameters['itR']) # Define bootstrapping iteration
    lambdA = parameters['wvl'] # Define wavelengths of IOP data
    # 1b. define parameters from user-supplied template
    biN = str(User_Dict['Depth Bin Size (m)']) # define depth bin
    STN = User_Dict['Station Name'] # specify station name
    now = dt.now() # Find current date and time
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S") # Convert date and time into a time stamp (dd/mm/yyyy HH:MM:SS)
    # 1c. update header metadata template with instrument-independent information
    headER['1'] += dt_string  # Add current date and time to header metadata
    headER['6'] += biN + ' m' # Add depth bin size to header metadata
    headER['7'] += itR # bootstrapping iteration
    ### Third, define user-supplied variables DEPENDENT on instrument type
    # 3a. define parameters from user-supplied template
    if inST == 'ac-s':
        # If user is processing ac-s data...
        pthWY = User_Dict['acs file*'] # Use the pathway of the original ac-s file as a place to store data
        XTRA_proc = User_Dict['c-interpolation'] # are attenuation wavelengths interpolated to absorption using spine?
        abs_str = '\ta' # Absorption marker for column headers
    elif inST == 'hs6': 
        # If user is processing hs6 data...
        pthWY = User_Dict['hs6 file*'] # Use the pathway of the original ac-s file as a place to store data
        XTRA_proc = User_Dict['Doxaran-Correction']
        abs_str = '\tbb' # Backscattering marker for column headers
    # 3b. update header metadata template with instrument-DEPENDENT information supplied by user
    headER['5'] += pthWY # add origianl IOP file (individual spectra)
    headER['8'] += XTRA_proc # specify whether or not attenuation wavelengths were interpolate/Doxaran correction was applied
    headER['10'] += str(len(lambdA)) # add number of wavelengths (channels) to the 11th header line
    ### Fourth, specify the header fields and wavelengths for the IOP data
    fieldS_c = '' # Empty character array to store attenuation column headers (fields) when bootstrapping ac-s data
    for l in lambdA:
        # for-loop takes each ac-s wavelength and converts it into an absorption and an attenuation column header
        headER['9'] += abs_str + str(l) # Absorption column header
        headER['10'] += '\t' + str(l) # add wavelength onto header line 11
        if inST == 'ac-s':
            # If user is processign ac-s data...
            fieldS_c += '\tc' + str(l) # Create attenuation column headers
    headER['9'] += fieldS_c # Add attenuation column headers onto header line 10
    ### Fifth, specify name and location of the binned IOP file name 
    File_BS = STN + '_' + inST + '_bin' + biN + '_itr' + itR + '.txt' # Create file name using user-specified information
    dash_IND = [w.start() for w in re.finditer('/', pthWY)] # Find forward slashes (specify folders) in the file pathway of unbinned IOPs
    dash_IND = dash_IND[-1] # Locate the last forward slash
    pthWY = pthWY[:dash_IND+1] # Separate pathway from file name
    diRECTory = pthWY + inST + '/' # Add a new segment to the unbinned IOPs file pathway. This will become a new folder
    createFolder(diRECTory) # Create a new folder for binned IOP file(s) (if it doesn't yet exist)
    ### Sixth, write the bootstrapped IOP file!
    with open(diRECTory+File_BS, 'w', newline='') as txtfile: # Write a new ascii file
        for i in np.arange(len(headER)):
            # for-loop writes the header metadata into the new file
            txtfile.write(headER[str(i)] + '\n')
        for nn in np.arange(len(BOOTSRAP_MATRIX[:,0])):
            # for-loop writes the binned IOP data into the new file
            if ~np.isnan(BOOTSRAP_MATRIX[nn,1]):
                # if the last line is not reached
                datUM = str(BOOTSRAP_MATRIX[nn,:].tolist())[1:-1] # Convert row of data from numpy array into comma-separated string. Remove brackets
                datUM = datUM.replace(', ','\t') # replace commas with tabs
                txtfile.write(datUM + '\n') # Write the data into the file
                keY = nn # this variable preserves last row of actual values e.g. (not nan values)
        BOOTSRAP_MATRIX[keY,0] = -1 # After data has been written into the file, replace the first column in the last line (depth) with -1
        datUM = str(BOOTSRAP_MATRIX[nn,:].tolist())[1:-1] # Re-convert last row of data to comma-separated string
        datUM = datUM.replace(', ','\t') # remove end of line characters
        txtfile.write(datUM + '\n') # re-write the last line of code into the file


In [147]:
def IOP_Bootstrapper(IOP_MATRIX,User_Dict,**parameters):
    """ Performs multiple single bootstrapping (with replacement) iteration for depth-binning IOPs. Compatible with ac-s and hs6.
    Inputs:
        IOP_MATRIX - first column MUST be depth
        User_Dict - python dictionary of user-supplied information (see UserTemplate)
    Keyword arguments:
        INST - instrument type acs/hs6
        wvl - numpy array of wavelengths
    Output:
        Boostrap_FileWriter is nested in this function. Therefore, the outputs are hydrolight-compatible IOP files"""
    INST = parameters['INST']  # Specify instrument used to collect data: acs or hs6
    lambdA = parameters['wvl'] # Specify wavelengths (used to write the file)
    BIN = float(User_Dict['Depth Bin Size (m)']) # Define user-input bin size for bootstrapping
    n = int(User_Dict['Bootstrap Iterations']) # Number of bootstrap iterations to be carried out
    Depth = IOP_MATRIX[:,0] # Find the depths for IOP spectra
    IOP_MATRIX = IOP_MATRIX[:,1:] # Eliminate depth column from IOP_MATRIX
    # Step 1: Create median depth bins for IOPs based on depth bin size
    d_min = np.floor(min(Depth)) # minimum depth
    d_max = np.ceil(max(Depth)) # maximum depth
    Depth_edgE = np.arange(d_min,d_max+BIN,BIN) # Calculate edges (boundaries) of each depth bin
    Depth_BINED = (Depth_edgE[1:]+Depth_edgE[:-1])/2 # # Calculate depth bin medians
    BinHalf = BIN/2 # Calculate half of depth bin
    # Step 2: Create bootstrapped ac-s files that are compatable with Hydrolight
    for k in np.arange(n):
        #  for-loop creates n bootstrap matrices, with 'n' corresponding to the number of 
        # bootstrapping iterations or "runs" that are specified by the user. Each iteration 
        # consists of one matrix of depth-binned IOPs (either absorption/attenuation or backscattering).
        BOOTSRAP_MATRIX = np.zeros([len(Depth_BINED),len(IOP_MATRIX[0,:])+1])*np.nan 
        # Create empty (nan) matrix for depth-binned bootstrapped IOP spectra
        BOOTSRAP_MATRIX[:,0] = Depth_BINED # Make the first column matrix binned depths
        for h,i in enumerate(Depth_BINED):
            # For each individual depth-bin, perform bootstrapping with replacement and calculate
            # mean spectrum of bootstrapped IOP spectra. Repeat for each depth bin.
            Depth_BIN_IND = np.where(np.logical_and(Depth >i-BinHalf, i+BinHalf>=Depth))
            # Find index of each IOP spectrum that falls between the given depth bin
            samplE_size = int(np.rint(len(Depth_BIN_IND[0]))) # determine indices of spectra that fall within the selected depth range
            if samplE_size > 0:
                # If there are IOP values at the specified depth range...
                BootSTRAP_IND = np.random.uniform(low=np.min(Depth_BIN_IND), high=np.max(Depth_BIN_IND), size=(samplE_size,1)) 
                # (above) randomly select numbers in range of the selected indices (these are ordered according to depth)
                BootSTRAP_IND = np.rint(BootSTRAP_IND) # Convert numbers to integers to correspond with indices
                BOOTSRAP_TEMP = np.zeros([len(BootSTRAP_IND),len(IOP_MATRIX[0,:])]) 
                # Create an empty matrix of zeros (above) that can fit spectra (indices x wavelengths)
                for keY,j in enumerate(BootSTRAP_IND):
                    # This for-loop will cycle through the matrix of zeros and assign a randomly-selected IOP spectra (e.g. indices) to each row 
                    BOOTSRAP_TEMP[keY,:] = IOP_MATRIX[int(j),:] # Assign randomly-selected IOP spectrum to a row
                BOOTSRAP_MATRIX[h,1:] = np.mean(BOOTSRAP_TEMP,axis=0) # Take mean of randomly-selected IOP spectra
        BOOTSRAP_MATRIX = np.around(BOOTSRAP_MATRIX,decimals=6) # Round all depth-binned IOPs to six decimal places
        Boostrap_FileWriter(BOOTSRAP_MATRIX,User_Dict,itR=k,INSTRUMENT=INST,wvl=lambdA) 
        # Write bootstrapped (depth-binned) IOP matrix into a Hydrolight-compatible data file

In [148]:
def HE53_batch_WRITER(User_Dict):
    """Creates a Hydrolight-compatible ascii file using depth-binned IOP data. File is output in a separate folder.
    This folder is placed in the same directory as the original file containing non-binned IOP data.
    Inputs:
        BOOTSRAP_MATRIX - Depth-binned IOP matrix
        User_Dict - Python dictionary containing user-specified inputs
    keyword arguments:
        itR - specify bootstrap iteration. Must be an integer.
        wvl - numpy array of IOP wavelengths.
        INSTRUMENT - specify whether IOP matrix is 'acs' or 'hs6'
    Outputs:
        Series of Hyrolight batch files that are ready to process"""
    ### First, import batch file template as batch_DATA.
    batch_DATA = headER_reader('HE53_Bootstrap_metadata1',key='batch')
    ### Second, assign station-specific information to the batch file template.
    batch_DATA['2'] += User_Dict['Station Name'] + '_bin' + User_Dict['Depth Bin Size (m)'] + '_itr' # Construct root file. Add to 3rd line
    batch_DATA['6'] += User_Dict['Chl Conc (mg/L)'] + ',' # Assign chlorophyll concentration to the 7th header line
    ##2a. Compute the hour and minute in decimal form
    timE = User_Dict['GMT (HH'].split(':') # Define time stamp (as string)
    hr = int(timE[2]) # Find the hour (GMT)
    hrFRACT = (int(timE[3])*60 + int(timE[4]))/3600 # Convert minutes and seconds into "hour" (decimal)
    batch_DATA['a1'] += str(hr+hrFRACT) + ', 0, 0' # Add decimal hour into line a1 of header template
    ## 2b. Compute day of year
    d = str(dt.strptime(User_Dict['Date (mm-dd-yyyy)'],"%m-%d-%Y").timetuple().tm_yday) # Calculate day of year from time stamp using datetime    
    ## 2b (continued). Add day of year, latitude, lontitude, atmospheric pressure, "5", % Humidity, Precipitable Water Content (cm), horizontal visibility, and windspeed to line a2 of header template line a3
    batch_DATA['a2'] += d + ', ' + User_Dict['Latitude****'] + ', ' + User_Dict['Longitude****'] + ', ' + User_Dict[ 'Pressure (inHg)'] + ', 5, ' + User_Dict['Humidity (%)'] + ', ' + User_Dict['Precipitable Water Content (cm)'] + ', ' + User_Dict['Horizontal Visibility (km)'] + ', ' + User_Dict['Wind Speed (m/s)'] + ', -99' 
    ## 2c. Add windspeed, water temperature, and salinity to header template a4 (below)
    batch_DATA['a3'] += User_Dict['Wind Speed (m/s)'] + ', -1.34, ' + User_Dict['Water Temperature (deg. C)'] +  ', ' + User_Dict['Salinity (PSU)']
    ## 2d. Add ac-s and hs6 root files (not iteration specific) to header template (lines b3 & b5 respectively). Include file pathways
    BIN = float(User_Dict['Depth Bin Size (m)'])  # Define depth bin size
    batch_DATA['b2'] += User_Dict['acs pathway**'] + 'acs\\' + User_Dict['Station Name'] + '_' + 'ac-s_bin' + str(BIN) + '_itr' # ac-s root file & pathway
    batch_DATA['b4'] += User_Dict['hs6 pathway**'] + 'hs6\\' + User_Dict['Station Name'] + '_' + 'hs6_bin' + str(BIN) + '_itr' # hs6 root file & pathway
    batch_DATA['b10'] += User_Dict['acs pathway**'] + User_Dict['Solar Irradiance file'] # Add solar irradiance file & pathway to header template
    ### Third, determine user-supplied wavelengths put them into lines (of ten) and add them to the header template
    wvl = User_Dict['Wavelengths***'] # Define user-supllied wavelengths
    ## 3a. Convert string to a numpy array of wavelengths
    if ':' in wvl:
        # If user supplied wavelengths in the format "minimum:increment:maximum"
        miN_wvl,miD_wvl,maX_wvl = wvl.split(':') # Split the string into three variables: minumum wavelength, increment, & masimum wavelength
        miN_wvl = float(miN_wvl) # Convert strings into floats (minimum)
        miD_wvl = float(miD_wvl) # wavelength increment
        maX_wvl = float(maX_wvl) # maximum wavelength
        lambdA = np.arange(miN_wvl,maX_wvl+miD_wvl,miD_wvl) # Convert into numpy array
    else:
        # If user supplied wavelengths as a list separated by commas
        lambdA = wvl.split(',') # split wavelengths into a list
        lambdA = np.asarray(lambdA,dtype=np.float) # Convert list into numpy array
    lambdA = np.sort(lambdA) # Sort wavelengths to be sure that they are in ascending order
    batch_DATA['23'] += str(len(lambdA)-1) # Tabulate number of wavelengths and add to 24th header template line
    ## 3b. Print order wavelengths into rows of 10 and write them into the header template. 
    ## Header template lines contain a 10 wavelengths or fewer. Wavelengths will be added between header template lines 23 and a1
    keY = 23 # Index variable for batch_DATA dictionary (with which to create new elements)
    for i,l in enumerate(lambdA):
        # for-loop adds wavelengths to template header one by one. After adding 10 wavelengths, a new line is started
        if np.remainder(i,10) == 0:
            # if 10 wavelengths are added to header template, start a new line of wavelengths
            keY += 1 # Increase index variable by 1
            batch_DATA[str(keY)] = '' # Add a new dictionary element to batch_DATA
        batch_DATA[str(keY)] += str(l) + ', ' # Add new wavelength and comma to dictionary element
    ### Fourth, determine user-supplied depth bins and add them to template header
    # 4a. Determine user-specified depth bins for Hydrolight to complete radiative transfer 
    # This will be the same value as used for IOP bootstrapping.
    max_Depth = float(User_Dict['Max Depth (m)']) # Define maximum depth
    Depth_BINS = np.arange(0,max_Depth+BIN,BIN) # Compute a numpy array for depth bins
    keY = 5 # Reset indexing variable to 5
    batch_DATA['a5'] += str(len(Depth_BINS)) + ', ' # Count all depth bins and add them to header template line a6
    for i,d in enumerate(Depth_BINS):
        # For-loop adds depth bins to the batch file template one by one. After adding 10 wavelength a new line is started
        if d - int(d) == 0:
            d = int(d)
        if np.remainder(i,10) == 0 and i != 0:
            # if 10 depth bins are added to header template, start a new line of depth bins
            keY += 1 # Increase indes variable by one
            batch_DATA['a' + str(keY)] = '' # Add a new dictionary element to batch_Data
        batch_DATA['a' + str(keY)] += str(d) + ', ' # Add new depth bin and comma to dictionary element
    ### Fifth, now that station-specific information has been added to batch file header template, generate a batch for 
    ### each bootstrap iteration
    ## 5a. Create a separate folder for batch files
    dash_IND = [w.start() for w in re.finditer('/', User_Dict['acs file*'])] # Find "/'s" for ac-s file string
    batch_pthwy = User_Dict['acs file*'][:dash_IND[-1]] + '/batch/' # Remove ac-s file from string and add '/batch/' as new folder
    createFolder(batch_pthwy) # if a folder named "batch" doesn't exist next to ac-s file, create one
    ## 5b. Create Hydrolight batch files for each set of bootstrapped IOP files
    n = int(User_Dict['Bootstrap Iterations']) # determine the number of bootstrapping iterations specified by user
    lettER = ['','a','b'] # This list of characters allows the program to cycle through different dictionary keys. 
    # These keys are denoted as blank, "a", and "b", depending on where in the batch file batch_DATA elements will be placed
    for i in np.arange(n):  
        # for-loop creates a new batch file each time it repeats itself. Within each for-loop iteration is a while loop
        # which is responsible for writing individual files. This while loop requires two indices (defined directly below)
        keY = 0 # Set numerical indexing component of batch_DATA key equal to zero
        L = 0   # Set letter index of batch_DATA lettER key equal to zero
        with open(batch_pthwy + 'I' + batch_DATA['2'] + str(i) + '.txt', 'w', newline='') as batchFID: 
            # Write a new batch file using iteration-specific root file. Add "I" at the beginning 
            # of the file name per Hydrolight protocol
            while 1:
                # While-loop runs through batch file header template line by line and writes them into the batch file in the correct order
                if L > 2:
                    # If all header template lines have been written into the batch file ...
                    break # Break the while-loop, close batch file being written, and begin a new batch file
                else:
                    # If not all header template lines have been written into the batch file ...
                    # Do the following....     
                    try: 
                        # If combined keY properly detects batch_DATA element
                        if lettER[L] + str(keY) == '2' or lettER[L] + str(keY) == 'b2' or lettER[L] + str(keY) == 'b4':
                            # If the header line contains a file that changes with each bootstrap iteration...
                            batchFID.write(batch_DATA[lettER[L] + str(keY)] + str(i) + '.txt\n') # Write a header line into batch file
                        else:
                            # If the header line contains a file that is the same for each bootstrap iteration...
                            batchFID.write(batch_DATA[lettER[L] + str(keY)] + '\n') # Write a header line into batch file
                        keY += 1 # increase numerical indexing component by 1 to allow for the next line to be written 
                    except:                            
                        # If the batch_DATA does not work for eather conditional statements...
                        keY = 0 # Reset numerical indexing component to zero
                        L += 1  # Increase letter index by 1. Move on to the next section of elements.
                        

In [149]:
## Execute Hydrolight53_Bootstrap

In [150]:
# Step 1: Read user-supplied template file and extract all relevant data
template_file = filedialog.askopenfilename() # Select template file
User_Dict = UserTemplate(template_file) # Place template information into python dictionary
acs_file = User_Dict['acs file*'] # Define file and pathway for ac-s (single spectra)
hs6_file = User_Dict['hs6 file*'] # Define file and pathway for hs6 (single spectra)

# Step 2: Prepare ac-s data for bootstrapping by reading it into variables containing
# absorption (Total_Abs), attenuation (Total_Atn), depth (Depth), and wavelength (lambdA)
if 'mat' in acs_file[-4:-1]:
    # If user specified a hdf5/mat file...
    Total_Abs,Total_Atn,Depth,lambdA = acs_HFD_LOADER(acs_file)
else:
    # If user specified a SEABASS ascii file...
    Total_Abs,Total_Atn,Depth,lambdA = SEABASS_TXT_LOADER(acs_file)

# Step 3: Prepare ac-s data for bootstrapping by ordering absorption and attenuation spectra 
# according to ascending depth
D_IND = np.argsort(Depth) # Find indices for ascending depths
Depth = Depth[D_IND]# Order depth according to ascending depth indices
Total_Abs = Total_Abs[D_IND,:] # Order absorption spectra according to ascending depth indices
Total_Atn = Total_Atn[D_IND,:] # Order attenuation spectra according to ascending depth indices

# Step 4: Combine absorption and attenuation spectra into a single matrix
l,w = Total_Abs.shape # Find dimensions of absorption and attenuation matrices (they should be the same size)
IOP_MATRIX = np.zeros([l,(w*2)+1])*np.nan # Create a matrix of nan values large enough to combine absorption and attenuation (horizontally)
IOP_MATRIX[:,0] = Depth # Insert depth as the first column of large matrix (IOP_MATRIX)
IOP_MATRIX[:,1:w+1] = Total_Abs # Insert absorption into the matrix 
IOP_MATRIX[:,w+1:] = Total_Atn # Insert attenuation into the matrix

# Step 5: Perform bootstapping with replacement on single ac spectra according to user-specified depth bin size. 
IOP_Bootstrapper(IOP_MATRIX,User_Dict,INST='ac-s',wvl=lambdA) # Create Hydrolight-compatible, depth-binned ac-s files for each bootstrap

# Step 6: Prepare hs6 data for bootstrapping by reading it into variables containing
# particulate backscattering (Total_hs), depth (Depth_hs), and wavelength (lambdA_hs)
if 'mat' in acs_file[-4:-1]:
    # If user specified a hdf5/mat file...
    Total_hs,blank,Depth_hs,lambdA_hs = acs_HFD_LOADER(hs6_file)
else:
    # If user specified a SEABASS ascii file...
    Total_hs,blank,Depth_hs,lambdA_hs = SEABASS_TXT_LOADER(hs6_file)

# Step 7: Prepare hs6 data for bootstrapping by ordering particulate baskcattering spectra according to ascending depth
D_IND_hs = np.argsort(Depth_hs) # Find indices for ascending depths
Depth_hs = Depth_hs[D_IND_hs] # Order depth according to ascending depth indices
Total_hs = Total_hs[D_IND_hs,:] # Order particulate backscattering spectra according to ascending depth indices

# Step 8: Prepare hs6 data for bootstrapping by ordering particulate backscattering spectra according to ascending wavlength
L_IND = np.argsort(lambdA_hs) # Find indices for ascending wavelengths
lambdA_hs = lambdA_hs[L_IND] # Order wavelengths according to ascending wavelength indices
Total_hs = Total_hs[:,L_IND] # Order particulate backscattering according to ascending wavelength indices

# Step 9: Combine particulate backscattering specra and depths into a matrix
l_hs, w_hs = Total_hs.shape # Find shape of particulate backscattering mtarix
IOP_MATRIX_hs = np.zeros([l_hs,w_hs+1])*np.nan # Create nan matrix for backscattering matrix with one additional column for depth
IOP_MATRIX_hs[:,0] = Depth_hs # Insert depth as the first column of large matrix
IOP_MATRIX_hs[:,1:w+1] = Total_hs # Insert particulate backscttering in the remaining columns of the nan matrix

# Step 10: Perform bootstapping with replacement on single ac spectra according to user-specified depth bin size. 
IOP_Bootstrapper(IOP_MATRIX_hs,User_Dict,INST='hs6',wvl=lambdA_hs) # Create Hydrolight-compatible, depth-binned ac-s files for each bootstrap

# Step 11: Create Hydrolight batch files consistent with instructions
HE53_batch_WRITER(User_Dict)